In [148]:
%matplotlib inline
import cv2
import numpy as np
from matplotlib import pyplot as plt

In [149]:
lower_red_1 = np.array([0, 150, 70])
higher_red_1 = np.array([8, 255, 255])
lower_red_2 = np.array([160, 150, 70])
higher_red_2 = np.array([180, 255, 255])
lower_yellow = np.array([20, 120, 75])
higher_yellow = np.array([60, 255, 255])

lower_blue = np.array([100, 100, 80])
higher_blue = np.array([135, 255, 255])

In [150]:
def setupLists(frame):
  # abhängig von der opencv version werden hier 2 oder 3 ausgabeparameter gebraucht
  contours, hierarchy = cv2.findContours(edge_detected_image, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE) # Edges to contours

  contour_list = []
  rect_list = []
  position_list = []

  for contour in contours:
      approx = cv2.approxPolyDP(contour,0.01*cv2.arcLength(contour,True),True) # Contour Polygons
      area = cv2.contourArea(contour)
      
      # Rechteck um Kontur
      rect = cv2.boundingRect(contour) # Polygon bounding rectangles
      x_rect,y_rect,w_rect,h_rect = rect
      x_rect +=  w_rect/2
      y_rect += h_rect/2
      area_rect = w_rect*h_rect
      
      # größere Entfernung erfordert das Herabsetzen der area > X Bedingung
      if ((len(approx) > 6) & (len(approx) < 20) & (area > 180) & (area_rect < (img_w*img_h)/5)) & (w_rect in range(h_rect-15,h_rect+15)): # Circle conditions
          contour_list.append(contour)
          position_list.append((x_rect,y_rect))
          rect_list.append(rect)
  
  position_list = list(set(position_list))

  return contour_list, rect_list, position_list

In [151]:
frame = cv2.imread(r"../imgs/andereposition_licht_approximately_40cm/camImage002.png")
img_h,img_w,_ = frame.shape

hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

blue_mask = cv2.inRange(hsv, lower_blue, higher_blue)

board_only = cv2.bitwise_and(frame, frame, mask=blue_mask)
cv2.imshow("board only", board_only)

board_only_gray = cv2.cvtColor(board_only, cv2.COLOR_BGR2GRAY)
cv2.imshow("gray board only", board_only_gray)

ret, thresh = cv2.threshold(board_only_gray, 10, 255, cv2.THRESH_BINARY)
cv2.imshow("Threshold Board", thresh)    

kernel_3 = np.ones((3,3), np.uint8)
kernel_5 = np.ones((5,5), np.uint8)

filled_noise = cv2.dilate(thresh, kernel_3, iterations=3)
cv2.imshow("fill small pixels", filled_noise)

noise_removed = cv2.erode(filled_noise, kernel_3, iterations=1)
cv2.imshow("remove small pixels", noise_removed)

# dilatation_dst = cv2.dilate(noise_removed, kernel_3, iterations=1)
# cv2.imshow("Dilate", dilatation_dst)
# # dilatation_dst = board_only_gray.copy()

# erode_dst = cv2.erode(dilatation_dst, kernel_3, iterations=1)
# cv2.imshow("Erode", erode_dst)
# # erode_dst = board_only_gray.copy()

edge_detected_image = cv2.Canny(noise_removed, 175, 200) 
cv2.imshow('Edge Detection', edge_detected_image)

contour_list, rect_list, position_list = setupLists(edge_detected_image)

img_circle_contours = frame.copy()

cv2.drawContours(img_circle_contours, contour_list,  -1, (0,255,0), thickness=1) # Display Circles

for rect in rect_list:
    x,y,w,h = rect
    cv2.rectangle(img_circle_contours,(x,y),(x+w,y+h),(0,0,255),1)

cv2.imshow('Circles Detected',img_circle_contours)

if (len(rect_list) > 0 and len(position_list) == 42):
    rows, cols = (6,7)
    mean_w = sum([rect[2] for r in rect_list]) / len(rect_list)
    mean_h = sum([rect[3] for r in rect_list]) / len(rect_list)

    position_list.sort(key = lambda x:x[1])
    position_list.sort(key = lambda x:x[0])

    # Find Colour Masks
    mask_red_1 = cv2.inRange(hsv, lower_red_1, higher_red_1)
    mask_red_2 = cv2.inRange(hsv, lower_red_2, higher_red_2)
    mask_red = mask_red_1 + mask_red_2
    

    img_red = cv2.bitwise_and(frame, frame, mask=mask_red)
    # cv2.imshow("Red Mask",img_red)

    mask_yellow = cv2.inRange(hsv, lower_yellow, higher_yellow)
    img_yellow = cv2.bitwise_and(frame, frame, mask=mask_yellow)
    # cv2.imshow("Yellow Mask",img_yellow)

    # Identify Colours
    grid = np.zeros((rows,cols))
    id_red = 1
    id_yellow = -1
    img_grid_overlay = frame.copy()
    img_grid = np.zeros([img_h,img_w,3], dtype=np.uint8)

    x_i = 0
    y_i = 0
    for i in range(0, len(position_list)):
        x = int(position_list[i][0])
        y = int(position_list[i][1])
        r = int((mean_h + mean_w)/5)


        img_grid_circle = np.zeros((img_h, img_w))
        cv2.circle(img_grid_circle, (x,y), r, (255,255,255),thickness=-1)
        img_res_red = cv2.bitwise_and(img_grid_circle, img_grid_circle, mask=mask_red)
        
        img_grid_circle = np.zeros((img_h, img_w))
        cv2.circle(img_grid_circle, (x,y), r, (255,255,255),thickness=-1)
        img_res_yellow = cv2.bitwise_and(img_grid_circle, img_grid_circle,mask=mask_yellow)
        # cv2.imshow("test", img_grid_circle)

        cv2.circle(img_grid_overlay, (x,y), r, (0,255,0),thickness=2)

        if img_res_red.any() != 0:
            grid[y_i][x_i] = id_red
            cv2.circle(img_grid, (x,y), r, (0,0,255),thickness=-1)
        elif img_res_yellow.any() != 0 :
            grid[y_i][x_i] = id_yellow
            cv2.circle(img_grid, (x,y), r, (0,255,255),thickness=-1)

        y_i += 1

        if y_i == 6:
            y_i = 0
            x_i += 1

    cv2.imshow('Img Grid Overlay',img_grid_overlay)
    cv2.imshow('Img Grid',img_grid)

while(True):
    k = cv2.waitKey(1)
    if k%256 == 27:
      # ESC pressed
      print("Escape hit, closing...")
      break

cv2.destroyAllWindows()

Escape hit, closing...
